In [ ]:
import os
import pandas as pd
from Dicom_RT_and_Images_to_Mask.src.DicomRTTool import DicomReaderWriter, pydicom
import SimpleITK as sitk
import numpy as np

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from ipywidgets import interact, fixed
from IPython.display import clear_output

# Callback invoked by the interact IPython method for scrolling through the image stacks of
# the two images (moving and fixed).
def display_images(fixed_image_z, moving_image_z, fixed_npa, moving_npa):
    # Create a figure with two subplots and the specified size.
    plt.subplots(1,2,figsize=(10,8))
    
    # Draw the fixed image in the first subplot.
    plt.subplot(1,2,1)
    plt.imshow(fixed_npa[fixed_image_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('fixed image')
    plt.axis('off')
    
    # Draw the moving image in the second subplot.
    plt.subplot(1,2,2)
    plt.imshow(moving_npa[moving_image_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('moving image')
    plt.axis('off')
    
    plt.show()

# Callback invoked by the IPython interact method for scrolling and modifying the alpha blending
# of an image stack of two images that occupy the same physical space. 
def display_images_with_alpha(image_z, alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[:,:,image_z] + alpha*moving[:,:,image_z] 
    plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()
    
# Callback invoked when the StartEvent happens, sets up our new data.
def start_plot():
    global metric_values, multires_iterations
    
    metric_values = []
    multires_iterations = []

# Callback invoked when the EndEvent happens, do cleanup of data and figure.
def end_plot():
    global metric_values, multires_iterations
    
    del metric_values
    del multires_iterations
    # Close figure, we don't want to get a duplicate of the plot latter on.
    plt.close()

# Callback invoked when the IterationEvent happens, update our data and display new figure.
def plot_values(registration_method):
    global metric_values, multires_iterations
    
    metric_values.append(registration_method.GetMetricValue())                                       
    # Clear the output area (wait=True, to reduce flickering), and plot current data
    clear_output(wait=True)
    # Plot the similarity metric values
    plt.plot(metric_values, 'r')
    plt.plot(multires_iterations, [metric_values[index] for index in multires_iterations], 'b*')
    plt.xlabel('Iteration Number',fontsize=12)
    plt.ylabel('Metric Value',fontsize=12)
    plt.show()
    
# Callback invoked when the sitkMultiResolutionIterationEvent happens, update the index into the 
# metric_values list. 
def update_multires_iterations():
    global metric_values, multires_iterations
    multires_iterations.append(len(metric_values))

In [ ]:
def return_MRN_dictionary(excel_path):
    df = pd.read_excel(excel_path, sheet_name='Refined')
    MRN_list, GTV_List, Ablation_list, Registered_list = df['MRN'].values, df['PreExam'].values,\
                                                         df['Ablation_Exam'].values, df['Registered'].values
    MRN_dictionary = {}
    for MRN, GTV, Ablation, Registered in zip(MRN_list, GTV_List, Ablation_list, Registered_list):
        Registered = str(Registered)
        if Registered != '1.0':
            continue
        add = True
        if type(GTV) is float or type(Ablation) is float:
            add = False
        if add:
            GTV = str(GTV)
            if GTV.startswith('CT'):
                if GTV.find(' ') == -1:
                    GTV = 'CT {}'.format(GTV.split('CT')[-1])
            Ablation = str(Ablation)
            if Ablation.startswith('CT'):
                if Ablation.find(' ') == -1:
                    Ablation = 'CT {}'.format(Ablation.split('CT')[-1])
            MRN_dictionary[MRN] = {'Primary': GTV, 'Secondary': Ablation}
    return MRN_dictionary

In [ ]:
base_export_path = r'H:\Deeplearning_Recurrence_Work\Dicom_Exports'
excel_path = r'\\mymdafiles\di_data1\Morfeus\BMAnderson\Modular_Projects\Liver_Local_Recurrence_Work' \
             r'\Predicting_Recurrence\RetroAblation.xlsx'
MRN_dictionary = return_MRN_dictionary(excel_path)

In [ ]:
for MRN in MRN_dictionary.keys():
    if MRN not in os.listdir(base_export_path):
        continue
    patient_dictionary = MRN_dictionary[MRN]
    primary = patient_dictionary['Primary']
    secondary = patient_dictionary['Secondary']
    primary_reader = DicomReaderWriter()
    secondary_reader = DicomReaderWriter()
    for root, directories, files in os.walk(os.path.join(base_export_path, MRN)):
        if 'Registration' in directories and primary in directories and secondary in directories:
            '''
            First, load in our registration
            '''
            registration_path = os.path.join(root, 'Registration')
            registration_file = [os.path.join(registration_path, i) for i in os.listdir(registration_path)][0]
            ds = pydicom.read_file(registration_file)
            registration_matrix = np.asarray(ds.RegistrationSequence[-1].MatrixRegistrationSequence[-1].
                                             MatrixSequence[-1].FrameOfReferenceTransformationMatrix)[:-4]
            registration_matrix = np.transpose(np.reshape(registration_matrix, (3, 4)))
            registration_matrix[-1, :] *= -1
            '''
            Next, our primary and secondary images, as sitkFloat32'''
            primary_path = os.path.join(root, primary)
            secondary_path = os.path.join(root, secondary)
            primary_reader.down_folder(primary_path)
            secondary_reader.down_folder(secondary_path)
            fixed_image = sitk.Cast(primary_reader.dicom_handle, sitk.sitkFloat32)
            moving_image = sitk.Cast(secondary_reader.dicom_handle, sitk.sitkFloat32)

In [ ]:
fixed =sitk.GetArrayFromImage(fixed_image)
moving = sitk.GetArrayFromImage(moving_image)

In [ ]:
for i in [fixed, moving]:
    i[i>200] = 200
    i[i<-100] = -100

In [ ]:
new_fixed = sitk.GetImageFromArray(fixed)
new_moving = sitk.GetImageFromArray(moving)

In [ ]:
new_fixed = sitk.Cast(new_fixed, sitk.sitkFloat32)
new_moving = sitk.Cast(new_moving, sitk.sitkFloat32)

In [ ]:
new_moving.SetDirection(moving_image.GetDirection())
new_moving.SetOrigin(moving_image.GetOrigin())
new_moving.SetSpacing(moving_image.GetSpacing())

In [ ]:
new_fixed.SetDirection(fixed_image.GetDirection())
new_fixed.SetOrigin(fixed_image.GetOrigin())
new_fixed.SetSpacing(fixed_image.GetSpacing())

In [ ]:
new_fixed = sitk.Cast(new_fixed, sitk.sitkFloat32)
new_moving = sitk.Cast(new_moving, sitk.sitkFloat32)

In [ ]:
fixed_image = new_fixed
moving_image = new_moving

In [ ]:
interact(display_images, fixed_image_z=(0,fixed_image.GetSize()[2]-1), moving_image_z=(0,moving_image.GetSize()[2]-1), fixed_npa = fixed(sitk.GetArrayViewFromImage(fixed_image)), moving_npa=fixed(sitk.GetArrayViewFromImage(moving_image)));

In [ ]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                     moving_image, 
                                                     sitk.AffineTransform(3), 
                                                     sitk.CenteredTransformInitializerFilter.GEOMETRY)

In [ ]:
k = sitk.AffineTransform(4)

In [ ]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                     moving_image, 
                                                     sitk.AffineTransform(3), 
                                                     sitk.CenteredTransformInitializerFilter.GEOMETRY)
initial_transform.SetParameters(registration_matrix.flatten())
moving_resampled = sitk.Resample(moving_image, fixed_image, initial_transform, sitk.sitkLinear, -1000.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2]-1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image), moving=fixed(moving_resampled));

In [ ]:
initial_transform.SetParameters(registration_matrix.flatten())

In [ ]:
registration_matrix

In [ ]:
list(np.transpose(np.reshape(registration_matrix, (3,4))))

In [ ]:
initial_transform.SetParameters((1., 0., 0., 0., 1., 0., 0., 0., 1., 21.4, -3.38, 636.417))

In [ ]:
affine_transform.SetParameters(np.transpose(registration_matrix).flatten())

In [ ]:
initial_transform.GetMatrix()

In [ ]:
registration_matrix

In [ ]:
fixed_image.GetOrigin()

In [ ]:
moving_image.TransformPhysicalPointToContinuousIndex(registration_matrix[:-1,-1])

In [ ]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_image, 
                                                      affine_transform, 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)
transform_point(initial_transform, point)

In [ ]:
moving_resampled = sitk.Resample(moving_image, fixed_image, affine_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2]-1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image), moving=fixed(moving_resampled));

In [ ]:
registration_method = sitk.ImageRegistrationMethod()

# Similarity metric settings.
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)

registration_method.SetInterpolator(sitk.sitkLinear)

# Optimizer settings.
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
registration_method.SetOptimizerScalesFromPhysicalShift()

# Setup for the multi-resolution framework.            
registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2,1,0])
registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

# Don't optimize in-place, we would possibly like to run this cell multiple times.
registration_method.SetInitialTransform(initial_transform, inPlace=False)

# Connect all of the observers so that we can perform plotting during registration.
registration_method.AddCommand(sitk.sitkStartEvent, start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, end_plot)
registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, update_multires_iterations) 
registration_method.AddCommand(sitk.sitkIterationEvent, lambda: plot_values(registration_method))

final_transform = registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), 
                                               sitk.Cast(moving_image, sitk.sitkFloat32))

In [ ]:

moving_resampled = sitk.Resample(moving_image, fixed_image, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image), moving=fixed(moving_resampled));